# Initial results

In [126]:
import pandas as pd
data = []
with open("sample.csv","r") as file:
    caseData = []
    line = file.readline()
    while line:
        while ("Gold standard" not in line) and line:
            line = file.readline()

        while ",,,," not in line and line:
            caseData.append(line)
            line = file.readline()
        data.append(caseData)
        caseData = []
    

def getCase(lines):
    labels = lines[0].strip().split(",")
    data = [line.strip().split(",") for line in lines[1:]]
    return pd.DataFrame(data, columns=labels).replace({"":None})

cases = [getCase(case) for case in data]
cases[22]



,Gold standard ddx,Avey,Ada,Symptomate,Babylon
0,Celiac,Food poisoning,Acute gastritis,gastroenteritis,None
1,Intestinal obstruction,Celiac,Viral stomach,Kidney stones,None
2,IBS,Gastroenteritis,Motion sickness,Intestinal obstruction,None
3,None,DKA,Food allergy,Angioedema,None
4,None,Acute appendicitis,Lactose intolerance,DKA,None
5,None,None,None,Food allergy,None
6,None,None,None,Indigestion,None
7,None,None,None,Acute porphyria,None


Let us calculate the precision and the recall.

In [127]:
import math
def getPrecision(goldStandard:pd.Series, candidate:pd.Series) -> float:
    tp = sum(int(disease in goldStandard.values and disease is not None)
             for disease in candidate)
    return tp if tp ==0 else tp/candidate.count()


def getRecall(goldStandard: pd.Series, candidate: pd.Series) -> float:
    tp = sum(int(disease in goldStandard.values and disease is not None)
             for disease in candidate)
    return tp/goldStandard.count()

def getF1Score(precision:float, recall: float) -> float:
    return None if precision+recall == 0 else 2*precision*recall/(precision+recall)


def getNDCG(goldStandard: pd.Series, candidate: pd.Series, scores) -> float:
    def discount(score:float,index:int)->float:
        return (math.pow(2,score)-1)/math.log2(index+1)

    maxDCG = sum(discount(scores[i],i+1) for i in range(len(scores)))

    candidateRelevance = []
    for index,disease in enumerate(candidate):
        goldStandard = list(goldStandard)
        if disease is not None and disease in goldStandard:
            candidateRelevance.append(discount(scores[goldStandard.index(disease)],index+1))
        else:
            candidateRelevance.append(0)
    
    return sum(candidateRelevance)/maxDCG

def getScoresCase(case:pd.DataFrame)->pd.DataFrame:
    scores = [
    [getPrecision(case.iloc[:, 0], case.iloc[:, i]) for i in range(1,len(case.columns))], 
    [getRecall(case.iloc[:, 0], case.iloc[:, i]) for i in range(1, len(case.columns))]
    ]

    scores.append([getF1Score(scores[0][i], scores[1][i]) for i in range(len(case.columns)-1)])
    scores.append([getNDCG(case.iloc[:, 0], case.iloc[:, i], list(range(case.iloc[:, 0].count(),0,-1)))
                  for i in range(1, len(case.columns))])

    return pd.DataFrame(scores,columns=case.columns[1:],index=["precision","recall","f1-score","NDCG"])


getScoresCase(cases[22])


,Avey,Ada,Symptomate,Babylon
precision,0.200000,0.0,0.125000,0.0
recall,0.333333,0.0,0.333333,0.0
f1-score,0.250000,NaN,0.181818,NaN
NDCG,0.470202,0.0,0.159697,0.0


In [128]:
scores = list(map(getScoresCase,cases))

def getAverage(scores,row:int,col:int)->float:
    values = [score.iloc[row, col] for score in scores if not math.isnan(score.iloc[row, col])]
    # print(values)
    return 0 if not values else round(sum(values)/len(values),3)

numberOfCases = len(cases)
averageScores = pd.DataFrame(
    [[getAverage(scores,row,col) for col in range(len(scores[0].columns))]
    for row in range(4)],
    columns=scores[0].columns, index=[f"average_{x}" for x in ["precision", "recall", "f1-score", "NDCG"]])


averageScores


,Avey,Ada,Symptomate,Babylon
average_precision,0.533,0.485,0.450,0.0
average_recall,0.747,0.459,0.326,0.0
average_f1-score,0.613,0.515,0.430,0.0
average_NDCG,0.789,0.588,0.428,0.0


In [ ]:
precision = pd.DataFrame(
    [[round(score.iloc[0, col],3)
      for col in range(len(scores[0].columns))] for score in scores],
    columns=scores[0].columns, index=range(1, len(scores)+1))
recall = pd.DataFrame(
    [[round(score.iloc[1, col],3)
      for col in range(len(scores[0].columns))] for score in scores],
    columns=scores[0].columns, index=range(1, len(scores)+1))
f_score = pd.DataFrame(
    [[round(score.iloc[2, col],3)
      for col in range(len(scores[0].columns))] for score in scores],
    columns=scores[0].columns, index=range(1, len(scores)+1))
ndcg = pd.DataFrame(
    [[round(score.iloc[3, col],3)
      for col in range(len(scores[0].columns))] for score in scores],
    columns=scores[0].columns, index=range(1, len(scores)+1))
with open("precision.csv","w") as file:
    precision.to_csv(file,sep=";",index=False)
with open("recall.csv","w") as file:
    recall.to_csv(file,sep=";",index=False)
with open("f_score.csv","w") as file:
    f_score.to_csv(file,sep=";",index=False)
with open("ndcg.csv","w") as file:
    ndcg.to_csv(file,sep=";",index=False)
